In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
data = pd.read_csv('C:\\Users\\Administrator\\Documents\\kaggle\\train.csv\\train.csv')

In [4]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

In [5]:
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

In [6]:
data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [7]:
Y_train

array([1, 5, 6, ..., 5, 7, 7], dtype=int64)

In [8]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

In [9]:
def ReLU(Z):
    return np.maximum(Z, 0)


In [10]:
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

In [11]:
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [12]:
def ReLU_deriv(Z):
    return Z > 0

In [13]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [14]:
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

In [15]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [16]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [17]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[3 2 3 ... 3 2 3] [1 5 6 ... 5 7 7]
0.0744390243902439
Iteration:  10
[2 6 6 ... 7 6 6] [1 5 6 ... 5 7 7]
0.10639024390243902
Iteration:  20
[2 6 6 ... 7 8 8] [1 5 6 ... 5 7 7]
0.17326829268292682
Iteration:  30
[2 2 7 ... 7 8 7] [1 5 6 ... 5 7 7]
0.23426829268292684
Iteration:  40
[2 3 7 ... 7 8 7] [1 5 6 ... 5 7 7]
0.3131951219512195
Iteration:  50
[1 3 7 ... 7 8 7] [1 5 6 ... 5 7 7]
0.39463414634146343
Iteration:  60
[1 3 6 ... 7 8 7] [1 5 6 ... 5 7 7]
0.45726829268292685
Iteration:  70
[1 3 6 ... 2 2 7] [1 5 6 ... 5 7 7]
0.5041707317073171
Iteration:  80
[1 3 6 ... 2 2 7] [1 5 6 ... 5 7 7]
0.5454390243902439
Iteration:  90
[1 3 6 ... 2 2 7] [1 5 6 ... 5 7 7]
0.5782926829268292
Iteration:  100
[1 3 6 ... 2 2 7] [1 5 6 ... 5 7 7]
0.6061463414634146
Iteration:  110
[1 3 6 ... 2 2 7] [1 5 6 ... 5 7 7]
0.6284146341463415
Iteration:  120
[1 3 6 ... 2 2 7] [1 5 6 ... 5 7 7]
0.6485609756097561
Iteration:  130
[1 3 6 ... 2 2 7] [1 5 6 ... 5 7 7]
0.6658536585365854
Iteration:  

In [18]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()